In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import requests

from dotenv import load_dotenv
from github import Github

In [ ]:
# Inserire il token generato su GitHub
token = "TOKEN"

# Definire l'URL dell'API di GitHub
url = "https://api.github.com/user"

# Definire l'header con l'autenticazione tramite token
headers = {
    "Authorization": f"Token {token}"
}

# Fare la richiesta all'API di GitHub
response = requests.get(url, headers=headers)

# Stampare la risposta ottenuta dall'API di GitHub
print(response.json())


In [ ]:
g = Github(token)
repos = g.get_repos()
users = []
num_repos = 2000
repos_to_shuffle = 3000
repos = list(repos[:repos_to_shuffle])

# To repeat the same random selection we can fix the seed
np.random.seed(33)
repos = np.random.choice(repos, num_repos, replace=False)

repos_dataset = []
# repos = ["r_"+str(i)]

In [ ]:
users = []
languages_list = []
for i, repo in enumerate(repos):
    try:
        r_id = "r_" + str(i)
        full_name = repo.full_name
        topics = repo.get_topics()
        print(f"Topics for {repo.name}: {topics}")
        n_star = repo.stargazers_count
        print (f"Stars for {repo.name}: {n_star}")
        n_fork = repo.forks_count
        print (f"Forks for {repo.name}: {n_fork}")
        languages = repo.get_languages()
        lan = list(languages.keys())
        total_lines = sum(languages.values())
        language_percentages = {k: v / total_lines * 100 for k, v in languages.items()}
        l_percentages = ['{:.2f}'.format(p) for p in language_percentages.values()]
        print(f"Languages for {repo.name}: {lan}")
        print(f"Languages for {repo.name}: {l_percentages}")
        
        repos_dataset.append({'id': r_id, 'name': full_name, 'topics': topics, 'n_star': n_star, 'n_fork': n_fork, 'languages': lan, '%_lang':l_percentages})

      #save languages in a list to create the nodes
        for l in languages:
            if l not in languages_list:
                languages_list.append(l)

        #contributors
        cs = repo.get_contributors()
        commits = repo.get_commits()
        for c in cs:
            c_commits = 0
            for commit in commits:
                if commit.author.login == c.login:
                    c_commits += 1
            c_data = {'repos': r_id, 'user': c.login, 'commits': c_commits}
            #print numeber of commits for each contributor
            print(f"Commits for {c.login}: {c_commits}")
            users.append(c_data)
        
    except Exception as e:
        print('Error for repo: ', repo.full_name, e)
        continue

In [ ]:
#create users.csv
users_df = pd.DataFrame(users)

users_df = users_df.groupby('user').agg({'commits': list, 'repos': list}).reset_index()
users_id = ['u_'+str(i) for i in range(len(users_df))]
users_df['id'] = users_id
users_df = users_df[['id', 'user', 'repos', 'commits']]

users_df.to_csv('dataset/users.csv', index=False)


In [ ]:
#create file languages.csv with the id and languages
languages_df = pd.DataFrame(languages_list)

languages_id = ['l_'+str(i) for i in range(len(languages_df))]
languages_df['id'] = languages_id
languages_df = languages_df[['id', languages_df.columns[0]]]   
languages_df.columns = ['id', 'language']

languages_df.to_csv('dataset/languages.csv', index=False)

In [ ]:
pd.DataFrame(repos_dataset).to_csv('dataset/repos.csv', index=False)

In [ ]:
repos_df = pd.DataFrame(repos_dataset)

languages_df = pd.read_csv('languages.csv')
language_id_dict = dict(zip(languages_df['language'], languages_df['id']))
repos_df['languages'] = repos_df['languages'].apply(lambda x: [language_id_dict[l] for l in x])

repos_df.to_csv('dataset/repos.csv', index=False)